<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [3]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

In [43]:
new_df=data_train.sample(50000)

Data preprocessing

In [44]:
new_df.head(3)

,id,article,highlights
276925,f2c51b0e0127c01f19ddc868ff1d49182d76a52b,by sophie jane evans a disabled chinese boy has been tied up by his family for six years after they struggled to cope with his aggressive behaviour xie guobiao 11 who suffers from a learning dis...,xie guobiao 11 has been tied up by family in daohui china since 2008 \nparents tie rope around his chest every day before attaching it to a pole \nthey only allow youngster out of the house for wa...
238374,c08982086dadf2d7809d986d05568471fe895752,baghdad iraq cnn iraqi leaders are applauding president obamas plan to withdraw most us troops from the country by august 2010 us troops will cede security measures to iraqi security forces inclu...,sunni vice president supports withdrawal of most troops by end of august 2010 \niraqi prime minister nuri almaliki says iraqi security forces are qualified to take over \nobama said up to 50000 su...
122592,2a71fe1c5039b8461ef97667247d448b15549af0,heartbeat recognition could soon be used to unlock doors start cars and even pay for your shopping thanks to new wearable software currently under development the nymi is a lightweight wearable wr...,the nymi authenticates user’s identity through the rhythm of their heartbeat \nthe wristband could replace pin numbers passwords bank cards and keys \nas soon as it is removed from the wearer’s bo...


In [6]:
data = pd.concat([data_train, data_test, data_validation ])
train_df = data

In [45]:
new_df.shape

(50000, 3)

In [46]:
sample=new_df.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
['by  chris hastings  published  1602 est 8 september 2012    updated  1704 est 8 september 2012  he has told how as a boy people dismissed him as thick because of his severe dyslexia which only makes it even more remarkable that jamie oliver has just been named as britain’s second biggestselling author since records began – with total book sales of £126\u2009million the essexborn television chef and school meals campaigner who has sold more than 10\u2009million cook books is ranked second only to harry potter author j\u2009k rowling cooking up a fortune jamie oliver pictured with his wife jools has come second in the list of the top 50 most valuable authors compiled by the bookseller  ms rowling’s seven books about the boy wizard have generated sales of more than\xa0 £2376\u2009millionthe newly released sales figures have been compiled by the bookseller – the publishing industry’s trade magazine it has drawn up lists of britain’s 50 most successful authors and the coun

Checking for Null values


In [47]:
new_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [48]:
new_df.duplicated(subset=['article','highlights']).sum()

0

In [49]:
new_df=new_df.drop_duplicates(subset=['article','highlights'])
new_df.shape

(50000, 3)

In [50]:
new_df.duplicated(subset=['article','highlights']).sum()

0

In [51]:
new_df['article'][:10]

276925    by  sophie jane evans  a disabled chinese boy has been tied up by his family for six years after they struggled to cope with his aggressive behaviour xie guobiao 11 who suffers from a learning dis...
238374    baghdad iraq cnn  iraqi leaders are applauding president obamas plan to withdraw most us troops from the country by august 2010 us troops will cede security measures to iraqi security forces inclu...
122592    heartbeat recognition could soon be used to unlock doors start cars and even pay for your shopping thanks to new wearable software currently under development the nymi is a lightweight wearable wr...
234695    by  daily mail reporter  published  1611 est 29 august 2013    updated  1011 est 30 august 2013  a terrified teenage couple were confronted by armed police on their first holiday abroad  over a ro...
185103    by  amanda williams  published  0745 est 28 november 2012    updated  1448 est 28 november 2012  louis walsh has today accepted 500000 euro £403500 in

In [52]:
new_df['article'] = new_df['article'].str.lower()
new_df['highlights'] = new_df['highlights'].str.lower()

new_df.head()

,id,article,highlights
276925,f2c51b0e0127c01f19ddc868ff1d49182d76a52b,by sophie jane evans a disabled chinese boy has been tied up by his family for six years after they struggled to cope with his aggressive behaviour xie guobiao 11 who suffers from a learning dis...,xie guobiao 11 has been tied up by family in daohui china since 2008 \nparents tie rope around his chest every day before attaching it to a pole \nthey only allow youngster out of the house for wa...
238374,c08982086dadf2d7809d986d05568471fe895752,baghdad iraq cnn iraqi leaders are applauding president obamas plan to withdraw most us troops from the country by august 2010 us troops will cede security measures to iraqi security forces inclu...,sunni vice president supports withdrawal of most troops by end of august 2010 \niraqi prime minister nuri almaliki says iraqi security forces are qualified to take over \nobama said up to 50000 su...
122592,2a71fe1c5039b8461ef97667247d448b15549af0,heartbeat recognition could soon be used to unlock doors start cars and even pay for your shopping thanks to new wearable software currently under development the nymi is a lightweight wearable wr...,the nymi authenticates user’s identity through the rhythm of their heartbeat \nthe wristband could replace pin numbers passwords bank cards and keys \nas soon as it is removed from the wearer’s bo...
234695,bbd8c5c2e9ea75b506538b7cfa77a721a4a4c2e4,by daily mail reporter published 1611 est 29 august 2013 updated 1011 est 30 august 2013 a terrified teenage couple were confronted by armed police on their first holiday abroad over a ro...,francesca stefanyszyn and ged sykes had no money to pay for damage \nthey had used the towels to lie on in a krakow park \nstaff at hotel called police over the lightly mudmarked towels
185103,7bca2c8963891ef0bb4ccc45b9f82d37bcc33417,by amanda williams published 0745 est 28 november 2012 updated 1448 est 28 november 2012 louis walsh has today accepted 500000 euro £403500 in libel damages from the sun in ireland after i...,walsh was falsely accused of groping a man outside a celebrity nightclub in 2011 \nhe said he would not have wished what happened to him on his worst enemy \nthe music mogul added he felt vindicat...


In [53]:
new_df['article']=new_df['article'].str.lower()

new_df['highlights']=new_df['highlights'].str.lower()

Converting to lower case

In [54]:
new_df.head()

,id,article,highlights
276925,f2c51b0e0127c01f19ddc868ff1d49182d76a52b,by sophie jane evans a disabled chinese boy has been tied up by his family for six years after they struggled to cope with his aggressive behaviour xie guobiao 11 who suffers from a learning dis...,xie guobiao 11 has been tied up by family in daohui china since 2008 \nparents tie rope around his chest every day before attaching it to a pole \nthey only allow youngster out of the house for wa...
238374,c08982086dadf2d7809d986d05568471fe895752,baghdad iraq cnn iraqi leaders are applauding president obamas plan to withdraw most us troops from the country by august 2010 us troops will cede security measures to iraqi security forces inclu...,sunni vice president supports withdrawal of most troops by end of august 2010 \niraqi prime minister nuri almaliki says iraqi security forces are qualified to take over \nobama said up to 50000 su...
122592,2a71fe1c5039b8461ef97667247d448b15549af0,heartbeat recognition could soon be used to unlock doors start cars and even pay for your shopping thanks to new wearable software currently under development the nymi is a lightweight wearable wr...,the nymi authenticates user’s identity through the rhythm of their heartbeat \nthe wristband could replace pin numbers passwords bank cards and keys \nas soon as it is removed from the wearer’s bo...
234695,bbd8c5c2e9ea75b506538b7cfa77a721a4a4c2e4,by daily mail reporter published 1611 est 29 august 2013 updated 1011 est 30 august 2013 a terrified teenage couple were confronted by armed police on their first holiday abroad over a ro...,francesca stefanyszyn and ged sykes had no money to pay for damage \nthey had used the towels to lie on in a krakow park \nstaff at hotel called police over the lightly mudmarked towels
185103,7bca2c8963891ef0bb4ccc45b9f82d37bcc33417,by amanda williams published 0745 est 28 november 2012 updated 1448 est 28 november 2012 louis walsh has today accepted 500000 euro £403500 in libel damages from the sun in ireland after i...,walsh was falsely accused of groping a man outside a celebrity nightclub in 2011 \nhe said he would not have wished what happened to him on his worst enemy \nthe music mogul added he felt vindicat...


In [17]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [55]:
new_df['article'] = new_df['article'].apply(remove_html_tags)

new_df['highlights'] = new_df['highlights'].apply(remove_html_tags)

In [56]:
new_df.head()

,id,article,highlights
276925,f2c51b0e0127c01f19ddc868ff1d49182d76a52b,by sophie jane evans a disabled chinese boy has been tied up by his family for six years after they struggled to cope with his aggressive behaviour xie guobiao 11 who suffers from a learning dis...,xie guobiao 11 has been tied up by family in daohui china since 2008 \nparents tie rope around his chest every day before attaching it to a pole \nthey only allow youngster out of the house for wa...
238374,c08982086dadf2d7809d986d05568471fe895752,baghdad iraq cnn iraqi leaders are applauding president obamas plan to withdraw most us troops from the country by august 2010 us troops will cede security measures to iraqi security forces inclu...,sunni vice president supports withdrawal of most troops by end of august 2010 \niraqi prime minister nuri almaliki says iraqi security forces are qualified to take over \nobama said up to 50000 su...
122592,2a71fe1c5039b8461ef97667247d448b15549af0,heartbeat recognition could soon be used to unlock doors start cars and even pay for your shopping thanks to new wearable software currently under development the nymi is a lightweight wearable wr...,the nymi authenticates user’s identity through the rhythm of their heartbeat \nthe wristband could replace pin numbers passwords bank cards and keys \nas soon as it is removed from the wearer’s bo...
234695,bbd8c5c2e9ea75b506538b7cfa77a721a4a4c2e4,by daily mail reporter published 1611 est 29 august 2013 updated 1011 est 30 august 2013 a terrified teenage couple were confronted by armed police on their first holiday abroad over a ro...,francesca stefanyszyn and ged sykes had no money to pay for damage \nthey had used the towels to lie on in a krakow park \nstaff at hotel called police over the lightly mudmarked towels
185103,7bca2c8963891ef0bb4ccc45b9f82d37bcc33417,by amanda williams published 0745 est 28 november 2012 updated 1448 est 28 november 2012 louis walsh has today accepted 500000 euro £403500 in libel damages from the sun in ireland after i...,walsh was falsely accused of groping a man outside a celebrity nightclub in 2011 \nhe said he would not have wished what happened to him on his worst enemy \nthe music mogul added he felt vindicat...


In [20]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [57]:
new_df['article'] = new_df['article'].apply(remove_punc)

new_df['highlights'] = new_df['highlights'].apply(remove_punc)

In [58]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_df['article'] = new_df['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
new_df['highlights'] = new_df['highlights'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
data_train.head(1)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,by associated press published 1411 est 25 october 2013 updated 1536 est 25 october 2013 the bishop of the fargo catholic diocese in north dakota has exposed potentially hundreds of church ...,bishop john folda of north dakota is taking time off after being diagnosed \nhe contracted the infection through contaminated food in italy \nchurch members in fargo grand forks and jamestown coul...


In [61]:
new_df.head(1)

,id,article,highlights
276925,f2c51b0e0127c01f19ddc868ff1d49182d76a52b,sophie jane evans disabled chinese boy tied family six years struggled cope aggressive behaviour xie guobiao 11 suffers learning disorder first held captive 2008 became nuisance hometown songyang ...,xie guobiao 11 tied family daohui china since 2008 parents tie rope around chest every day attaching pole allow youngster house walks rope lead xie learning disorder reportedly caused trouble vill...


In [23]:
!pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.3 MB/s eta 0:00:00


In [24]:
from nltk import word_tokenize,sent_tokenize

In [26]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [28]:
data_train.shape

(284015, 3)

In [29]:
data_test.shape

(11490, 3)

In [30]:
#new_df=data_train.sample(50000)

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(new_df))

#convert text sequences into integer sequences
new_df    =   x_tokenizer.texts_to_sequences(new_df)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
new_df    =   pad_sequences(new_df,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1